In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import timm
from EFF3D.MRI import *
from fastai.vision.all import *
from EFF3D.res import resnet10
import torch.nn as nn

In [ ]:
df=pd.read_csv('../nonorm/ADNCold.csv')
# path=Path('/home/staff/xin/Downloads/newMRI/ADtrain')

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MriTensorImage.create)),CategoryBlock),
               get_x=ColReader('name'),
               get_y=ColReader('label'),
               splitter=RandomSplitter(valid_pct=0.2,seed=2),
              )

dls=db.dataloaders(source=df, bs=8, num_workers=4)

In [ ]:
encoder=resnet10(shortcut_type='B')
encoder=nn.DataParallel(encoder)
weight=torch.load('../nonorm/models/resnet_10_23dataset.pth')
encoder.load_state_dict(weight['state_dict'],strict=False)
encoder=nn.Sequential(*list(encoder.module.children()))
head=nn.Sequential(nn.AdaptiveMaxPool3d(1), Flatten(), nn.Linear(512, 2))
apply_init(head)
model=nn.Sequential(encoder,head)

/home/staff/xin/Documents/fastai2/SMRIEFF3D/EFF3D/res.py:174: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [ ]:
learn=Learner(dls,model=model,loss_func=LabelSmoothingCrossEntropy(),metrics=accuracy).to_non_native_fp16()

In [ ]:
learn.fit_one_cycle(200,1e-4,cbs=[CSVLogger(fname='medres10max.csv',append=False),SaveModelCallback(monitor='accuracy',fname='medres10max')])

epoch,train_loss,valid_loss,accuracy,time
0,2.208840,0.992757,0.556075,07:51
1,0.922655,0.741545,0.570093,07:50


Better model found at epoch 0 with accuracy value: 0.5560747385025024.
Better model found at epoch 1 with accuracy value: 0.5700934529304504.
